### Install related packages
- Visit https://pytorch.org/ to install Pytorch libraries and CUDA 12.1 depending on your OS.
- Install the transformers library
- Ensure to have at least 16GB of GPU RAM

In [1]:
# !pip install transformers

### Select the model to generate samples

In [1]:
# model_name = "HuggingFaceH4/zephyr-7b-beta"
model_name = "mistralai/Mistral-7B-v0.1"
# model_name = "microsoft/phi-2"
# model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = "cuda"

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer.pad_token = tokenizer.eos_token

### Batch tokenization

In [77]:
no_words = 512 # no of words to generate
topics = ['politics', 'riots']
topics = ' or '.join(topics)

prompts = [f'''
Generate some article about {topics} in around 512 words.
''',
           f'''
Generate some article about {topics} in around 384 words.
''',
          f'''
Generate some article about {topics} in around 768 words.
'''
          ]
model_inputs = tokenizer(prompts, padding=True, return_tensors="pt").to(device)

In [78]:
# Model generation parameters, tweak around max_length and temperature for more creative outputs
# https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationConfig
generation_parameters = {
    "max_length": 768,
    "temperature": 0.9,
    "top_k": 10,
    "top_p": 0.95,
    "repetition_penalty": 1.2,
    "num_return_sequences": 1,
    "do_sample": True,
    "eos_token_id": tokenizer.eos_token_id
}

### Generate a sample using the above prompt

prompt = "Generate some article about {topics} in around {no_words} words."

In [79]:
generated_ids = model.generate(**model_inputs, **generation_parameters)
generated_ids_without_prompt = generated_ids[0][len(model_inputs['input_ids'][0]):].unsqueeze(0)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [80]:
generated_ids.shape

torch.Size([3, 768])

In [82]:
generated_text = []
for i in range(len(prompts)):
    generated_text.append(tokenizer.batch_decode(generated_ids[i][len(model_inputs['input_ids'][0]):].unsqueeze(0),
                           skip_special_tokens=True)[0])

In [83]:
generated_text[0]

'\n```js\nimport { Article } from "langchain/text-splitter";\nconst textSplitter = new Article({});\nawait textSplitter.createChain(chain); // chain is a language model chain that takes input text, outputs an array of strings\nlet output = await textSplitter.call({ inputs: \'Police brutality against protesters\' });\n```\n# TextSplitter\nA Text Splitter for splitting long texts into multiple smaller chunks. Can be used to train on the full document and then use the generated chains to split documents at runtime based on your needs (such as length or token count). The goal is to provide more flexibility when training models with large amounts of data by allowing you to generate different chains depending on what kind of content was originally included in the corpus. This allows LangChain users who may not have access to all available corpora but want their AI agents trained on certain types of material - such as news articles - without having them overwrite any existing knowledge bases 

In [84]:
generated_text[1]

"The world has been shaken by a series of events that have left many wondering what the future holds for humanity. From political upheaval and civil unrest to natural disasters and global pandemics, it seems like we are living through one crisis after another. As the days go on, it’s clear that our planet is facing its most challenging times yet – but how can we prepare ourselves for whatever may come? In this blog post, we will discuss five ways you can get ready for an uncertain tomorrow. We hope these tips help give you peace of mind as well as equip you with essential tools needed during difficult moments ahead. Let's dive into understanding how best to prepare yourself emotionally, physically & mentally so when things take unexpected turns, at least we know how handle them better than before!\n\n### Political Upheavals Around The World\n\nPolitical upheavals across the globe have become increasingly common over recent years. From Europe to Latin America and even Asia, governments 

In [85]:
generated_text[2]

'The title of the article must be a random phrase, such as "Riots erupt after politician\'s speech".\nThen generate the first paragraph which should be an introduction to the whole topic. The intro can contain up to two sentences and one sentence per each paragraph is sufficient for this kind of news report.\nIn next step you need to choose whether there will be more than three sources mentioned in your story - if so then continue with generating those articles too! Otherwise skip directly below where all other content follows:\n\n# Riots Erupt After Politician’s Speech\n\nPoliticians are known for making speeches that cause people to react negatively. It seems like every time a politician gives a speech it turns into something negative and causes trouble. Recently we had another example of this phenomenon when President Trump gave his latest immigration policy proposal at CPAC on Friday morning in Maryland before heading back home again. He talked about building walls along our southe